In [ ]:
# ==========================================
# PHASE I: DETECTING
# ==========================================
import os
import json
import shutil
import getpass
import pandas as pd
from google.colab import files

# 1. SETUP CREDENTIALS

# IMPORTANT: Replace these with your actual Kaggle credentials before running!
kaggle_creds = {
    "username": "YOUR_USERNAME_HERE",
    "key": "YOUR_API_KEY_HERE"
}
with open('kaggle.json', 'w') as f:
    json.dump(kaggle_creds, f)

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("Keys set up.")

# 2. DOWNLOAD DATASET
print("\n Downloading Dataset...")
if not os.path.exists("/content/MOT17_Dataset"):
    !kaggle datasets download -d wenhoujinjust/mot-17
    !unzip -q mot-17.zip -d /content/MOT17_Dataset
print("Data Downloaded & Unzipped.")

# 3. PREPROCESSING
base_path = "/content/MOT17_Dataset/MOT17"
output_path = "/content/yolo_mot17"
train_sequences = ['MOT17-02-FRCNN', 'MOT17-04-FRCNN', 'MOT17-05-FRCNN', 'MOT17-09-FRCNN', 'MOT17-10-FRCNN', 'MOT17-11-FRCNN', 'MOT17-13-FRCNN']

# Clearing old folder to be safe
if os.path.exists(output_path): shutil.rmtree(output_path)

for split in ['train', 'val']:
    os.makedirs(f"{output_path}/images/{split}", exist_ok=True)
    os.makedirs(f"{output_path}/labels/{split}", exist_ok=True)

def convert_to_yolo_bbox(bbox, w, h):
    x_center = (bbox[0] + bbox[2] / 2) / w
    y_center = (bbox[1] + bbox[3] / 2) / h
    width = bbox[2] / w
    height = bbox[3] / h
    return x_center, y_center, width, height

print("\n Starting Preprocessing...")

for seq in train_sequences:
    seq_path = os.path.join(base_path, 'train', seq)
    img_dir = os.path.join(seq_path, 'img1')
    gt_file = os.path.join(seq_path, 'gt', 'gt.txt')
    ini_file = os.path.join(seq_path, 'seqinfo.ini')

    if not os.path.exists(ini_file): continue
    with open(ini_file, 'r') as f:
        info = f.read().splitlines()
        img_width = int([l for l in info if 'imWidth' in l][0].split('=')[1])
        img_height = int([l for l in info if 'imHeight' in l][0].split('=')[1])

    with open(gt_file, 'r') as f: lines = f.readlines()

    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        class_id = int(data[7])
        visibility = float(data[8])

        if class_id == 1 and visibility > 0.2:
            bbox = [float(data[2]), float(data[3]), float(data[4]), float(data[5])]
            yolo_bbox = convert_to_yolo_bbox(bbox, img_width, img_height)

            img_name = f"{seq}_{frame_id:06d}.jpg"
            label_name = f"{seq}_{frame_id:06d}.txt"
            split = 'train' if frame_id < (len(os.listdir(img_dir)) * 0.8) else 'val'

            with open(f"{output_path}/labels/{split}/{label_name}", 'a') as outfile:
                outfile.write(f"0 {yolo_bbox[0]} {yolo_bbox[1]} {yolo_bbox[2]} {yolo_bbox[3]}\n")

            src_img = os.path.join(img_dir, f"{frame_id:06d}.jpg")
            dst_img = os.path.join(output_path, 'images', split, img_name)
            if not os.path.exists(dst_img):
                shutil.copy(src_img, dst_img)

print("Preprocessing Done!")

# 4. CREATE CONFIG FILE
yaml_content = f"""
train: /content/yolo_mot17/images/train
val: /content/yolo_mot17/images/val
nc: 1
names: ['pedestrian']
"""
with open('/content/data.yaml', 'w') as f:
    f.write(yaml_content)
print("data.yaml created.")

# 5. INSTALL & TRAIN
print("\n Installing YOLOv5...")
if os.path.exists("/content/yolov5"): shutil.rmtree("/content/yolov5")
!git clone https://github.com/ultralytics/yolov5 > /dev/null
%cd /content/yolov5
!pip install -r requirements.txt > /dev/null

print("\n Starting Training Case 1 (YOLOv5s)...")
!python train.py --img 640 --batch 16 --epochs 5 --data /content/data.yaml --weights yolov5s.pt --name mot17_yolov5s --exist-ok

print("\n Starting Training Case 2 (YOLOv5m)...")
!python train.py --img 640 --batch 16 --epochs 5 --data /content/data.yaml --weights yolov5m.pt --name mot17_yolov5m --exist-ok

print("\n Starting Training Case 3 (YOLOv5m Frozen)...")
!python train.py --img 640 --batch 16 --epochs 5 --data /content/data.yaml --weights yolov5m.pt --freeze 10 --name mot17_yolov5m_frozen --exist-ok

# 6. EXTRACT IMPORTANT RESULTS & DOWNLOAD
print("\n" + "="*50)
print("FINAL RESULTS (Copy these to your Report)")
print("="*50)
print(f"{'Model':<25} | {'mAP@0.5':<10} | {'Precision':<10} | {'Recall':<10}")
print("-" * 65)

models = ['mot17_yolov5s', 'mot17_yolov5m', 'mot17_yolov5m_frozen']
runs_dir = "/content/yolov5/runs/train"

for model_name in models:
    path = os.path.join(runs_dir, model_name, "results.csv")
    if os.path.exists(path):
        df = pd.read_csv(path)
        df.columns = df.columns.str.strip()
        best_row = df.loc[df['metrics/mAP_0.5'].idxmax()]
        print(f"{model_name:<25} | {best_row['metrics/mAP_0.5']:.4f}     | {best_row['metrics/precision']:.4f}     | {best_row['metrics/recall']:.4f}")
    else:
        print(f"{model_name:<25} | NOT FOUND")
print("-" * 65)

print("\n Zipping Important Results (Images, Logs, Weights)...")
output_zip = "/content/Final_MOT17_Results.zip"
# This zips the entire 'runs' folder so you get images, graphs, and the 'best.pt' file
os.system(f"zip -r {output_zip} {runs_dir}")

file_size_mb = os.path.getsize(output_zip) / (1024 * 1024)
print(f"Downloading zip file ({file_size_mb:.2f} MB)...")
files.download(output_zip)


In [ ]:
# ==========================================
# PHASE II: TRACKING (DeepSORT / BoT-SORT)
# ==========================================
import os
from google.colab import files
from ultralytics import YOLO

# --- SETUP ---
print("Installing Tracking Libraries...")
!pip install -q ultralytics

print("\n Downloading Sample Video...")
%cd /content
if not os.path.exists("mot_sample.mp4"):
    !wget -q -O mot_sample.mp4 https://github.com/intel-iot-devkit/sample-videos/raw/master/people-detection.mp4

# --- LOAD CUSTOM MODEL ---
# We use the best weights from Phase I
custom_model_path = "/content/yolov5/runs/train/mot17_yolov5m_frozen/weights/best.pt"

print(f"Loading Custom Model: {custom_model_path}")
if os.path.exists(custom_model_path):
    model = YOLO(custom_model_path)
else:
    print("Custom model not found. Using standard YOLOv8n as fallback.")
    model = YOLO('yolov8n.pt')

# --- RUN TRACKING ---
# classes=[0] ensures we only track People (Class 0) and ignore background noise like TVs
print("Running Tracker...")
results = model.track(
    source="mot_sample.mp4",
    save=True,
    classes=[0],       # Filter: Person Only
    conf=0.5,          # Confidence Threshold
    project="/content/runs",
    name="mot_tracking_results",
    exist_ok=True
)

print("\n Tracking Complete!")

# --- DOWNLOAD RESULT ---
import glob
video_files = glob.glob("/content/runs/mot_tracking_results/*.avi") + glob.glob("/content/runs/mot_tracking_results/*.mp4")

if video_files:
    print(f" Downloading: {video_files[0]}")
    files.download(video_files[0])
else:
    print(" Output video not found.")
